compare calibration result and simulation result in the instant loop detector

In [29]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-01'
detector = 'w2e_in'
number = 100
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [32]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [19]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,w2e_in,1.577837e+09,enter,0_w2e_in,9.05,5.00,DEFAULT_VEHTYPE,None
3,w2e_in,1.577837e+09,enter,1_w2e_in,8.90,5.00,DEFAULT_VEHTYPE,1.74
5,w2e_in,1.577837e+09,enter,2_w2e_in,8.90,5.00,DEFAULT_VEHTYPE,1.28
8,w2e_in,1.577837e+09,enter,3_w2e_in,11.30,5.00,DEFAULT_VEHTYPE,9.71
10,w2e_in,1.577837e+09,enter,4_w2e_in,7.71,5.00,DEFAULT_VEHTYPE,20.46
...,...,...,...,...,...,...,...,...
224,w2e_in,1.577838e+09,enter,95_w2e_in,10.52,5.00,DEFAULT_VEHTYPE,7.41
226,w2e_in,1.577838e+09,enter,96_w2e_in,10.53,5.00,DEFAULT_VEHTYPE,33.52
228,w2e_in,1.577838e+09,enter,97_w2e_in,7.96,5.00,DEFAULT_VEHTYPE,13.56
230,w2e_in,1.577838e+09,enter,98_w2e_in,8.26,5.00,DEFAULT_VEHTYPE,1.68


In [33]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_w2e_in,1.577837e+09,11.448342,1.374351,1577836811,1577836773,max,9.166667,0.08,2.281676
1,1_w2e_in,1.577837e+09,10.842220,1.301587,1577836814,1577836774,max,7.500000,0.09,3.342220
2,2_w2e_in,1.577837e+09,10.467541,1.256608,1577836815,1577836775,max,8.888889,2.10,1.578652
3,3_w2e_in,1.577837e+09,10.775685,1.293600,1577836826,1577836786,max,11.111111,0.09,-0.335426
4,4_w2e_in,1.577837e+09,7.675853,0.921471,1577836847,1577836791,max,7.777778,0.13,-0.101925


In [34]:
print(len(data), len(df_instant_out))

100 100


compare time error and speed error in loop detector between simulation result and calibration result

In [ ]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_w2e_in,1.577837e+09,11.45,1.577837e+09,11.448342,1577836811,9.166667
1,1_w2e_in,1.577837e+09,10.84,1.577837e+09,10.842220,1577836814,7.500000
2,2_w2e_in,1.577837e+09,10.47,1.577837e+09,10.467541,1577836815,8.888889
3,3_w2e_in,1.577837e+09,10.78,1.577837e+09,10.775685,1577836826,11.111111
4,4_w2e_in,1.577837e+09,7.68,1.577837e+09,7.675853,1577836847,7.777778


In [38]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.8128999876976013)

In [39]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.8248999905586243)

In [40]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(2.0809711255951946)

In [41]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(2.0943444444444443)

In [127]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [128]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-3.359999895095825 8.940000057220459
-1.9100000858306885 10.579999923706055


In [129]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-6.666666666666666 6.111111111111111
-4.757534516970681 6.108111111111111


In [130]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

environ({'USER': 'kaveh', 'SSH_CLIENT': '50.98.120.254 52394 22', 'LC_TIME': 'sv_SE.UTF-8', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '1', 'MOTD_SHOWN': 'pam', 'HOME': '/home/kaveh', 'OLDPWD': '/home/kaveh/.vscode-server', 'LC_MONETARY': 'sv_SE.UTF-8', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1007/bus', 'LOGNAME': 'kaveh', '_': '/home/kaveh/calib_venv/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '8314', 'VSCODE_CLI_REQUIRE_TOKEN': 'ae03dc89-2af2-452f-9f46-712bcd09a747', 'PATH': '/home/kaveh/calib_venv/bin:/home/kaveh/.vscode-server/cli/servers/Stable-848b80aeb52026648a8ff9f7c45a9b0a80641e2e/server/bin/remote-cli:/home/kaveh/.local/bin:/home/kaveh/anaconda3/bin:/home/kaveh/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT_FOLDER': '/home/kaveh/.vscode-server', 'LC_ADDRESS': 'sv_SE.UTF-8', 'XDG_RUNTIME_DIR': '/run/user/1007', 'SSL

In [1]:
import os
os.environ["HORNSGATAN_HOME"]


'/home/kaveh/Hornsgatan/'